In [1]:
"""
This code only implements CNN model with batch size 64 and get the results of confusion matrix based on test set
and also implements the imagine processing before CNN model. As for other CNN models listed in the report, you 
only need to change the parameter in the layer. For adding intercepts, use bias_initializer="ones"; for regularization,
use kernel_regularizer=regularizers.l2(0.01)(for L2 penalty) and kernel_regularizer=regularizers.l1(0.01)(for L1 penalty);
for average global pooling, use cnn.add(GlobalAveragePooling2D()) instead of cnn.add(Flatten())
"""





import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import warnings
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns


#Get the train data and test data
train=pd.read_csv('../train_update.csv', sep=",")
test=pd.read_csv('../test.txt', sep=" ",header=None)
train.drop('Unnamed: 0', axis=1,inplace=True)
test.columns=["patient_id","filename","labels","sourses"]


dstpath='..'
path_train=[]
for line in train["filename"]:
    path_train.append(dstpath+'/'+line)
path_train=pd.DataFrame({'filename':path_train})
labels=train["classes"].copy()
train_df=pd.concat([path_train,labels],axis=1)
train_df.columns=["filename","labels"]

#set the features of imagine
img_height = 300
img_width = 300
batch_size=64
img_size=(img_height, img_width)

# Create Image Data Generator for Train Set
image_gen = ImageDataGenerator(
                                  rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True,          
                               )

# Create Image Data Generator for Test/Validation Set
test_data_gen = ImageDataGenerator(rescale = 1./255)

def scalar(img):
    return img/127.5-1  # scale pixel between -1 and +1
train=image_gen.flow_from_dataframe(train_df, x_col='filename', y_col='labels', 
                                    target_size=img_size, color_mode='grayscale'
                                    ,class_mode='binary',batch_size=batch_size)


test=test_data_gen.flow_from_dataframe(test,'../', 
                                       x_col='filename', y_col='labels', target_size=img_size, 
                                       color_mode='grayscale',
                                       shuffle=False, 
#setting shuffle as False just so we can later compare it with predicted values without having indexing problem 
                                       class_mode='binary',batch_size=batch_size)

#plot the x-ray figures
plt.figure(figsize=(12, 12))
for i in range(0, 10):
    plt.subplot(2, 5, i+1)
    for X_batch, Y_batch in train:
        image = X_batch[0]        
        dic = {0:'negative', 1:'positive'}
        plt.title(dic.get(Y_batch[0]))
        plt.axis('off')
        plt.imshow(np.squeeze(image),cmap='gray',interpolation='nearest')
        break
plt.tight_layout()
plt.show()



#Build the CNN model
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Flatten())
cnn.add(Dense(activation = 'relu', units = 128,kernel_regularizer=regularizers.l2(0.01)))
cnn.add(Dense(activation = 'relu', units = 64,kernel_regularizer=regularizers.l2(0.01)))
cnn.add(Dense(activation = 'sigmoid', units = 1,kernel_regularizer=regularizers.l2(0.01)))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
cnn.summary()

#Set the stopping rules in each epoch
early = EarlyStopping(mode="min", patience=3)
learning_rate_reduction = ReduceLROnPlateau(patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
callbacks_list = [ early, learning_rate_reduction]

#Set the weight to each class to keep balance of samples
weights = compute_class_weight('balanced', np.unique(train.classes), train.classes)
cw = dict(zip( np.unique(train.classes), weights))
print(cw)
warnings.filterwarnings("ignore")

#Fit the cnn model
history=cnn.fit(train,epochs=10, class_weight=cw, callbacks=callbacks_list)

#Put the test set into the model
test_accu = cnn.evaluate(test)
print('The testing accuracy is :',test_accu[1]*100, '%')
preds = cnn.predict(test,verbose=1)
predictions = preds.copy()
predictions[predictions <= 0.5] = 0
predictions[predictions > 0.5] = 1

#Get the confusion matrix
cm = pd.DataFrame(data=confusion_matrix(test.classes, predictions, labels=[0, 1]),index=["Actual Negative", "Actual Positive"],
columns=["Predicted Negative", "Predicted Positive"])

#Draw the confusion matrix and save the figure
sns_plot=sns.heatmap(cm,annot=True,fmt="d")
fig = sns_plot.get_figure()
fig.savefig('../cnn_method.png')